In [1]:

import regex as re
import pandas as pd
import numpy as np
from termcolor import colored
from urllib.request import urlopen
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [9]:
%pwd 
%cd c:\\Users\\Lenovo\\Flask apps\\House-price-prediction\\

c:\Users\Lenovo\Flask apps\House-price-prediction


In [85]:
MAIN_DF_PATH = r'Dataset\calgary_cleaned_rentals_dataset.csv'
df = pd.read_csv(MAIN_DF_PATH)

In [86]:

df.dropna(subset=['price', 'sq_feet'], inplace=True)
df.reset_index(inplace=True, drop=True)

assert len(df.rented.unique()) ==1
df.drop(columns=['rented'], inplace=True)


In [87]:
# handling second rentals
def handle_second_rentals(df, info_cols = ['price', 'sq_feet', 'beds', 'baths']):

    df_second = df[(~df[info_cols[0] + '2'].isna())&(~df[info_cols[1]+'2'].isna())]

    df_second = df_second.drop(columns=info_cols)
    for col in info_cols:
        df_second = df_second.rename(columns={col + '2': col})

    df = df.drop(columns=info_cols)
    return df, df_second

In [88]:
df, df_second = handle_second_rentals(df, info_cols = ['price', 'sq_feet', 'beds', 'baths'])

In [91]:
df_total = pd.concat([df, df_second], axis=0)
df_total.reset_index(inplace=True, drop=True)

In [92]:
len(df_total)

2213

In [90]:
len(df)

2033

In [68]:
df_second

,availability,a,city,community,latitude,longitude,link,type,cats,dogs,sq_feet,price,beds,baths,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
7,Immediate,2000-01-01,Calgary,Braeside,50.962467,-114.109704,/ab/calgary/rentals/apartment/1-bedroom/braeside/pet-friendly/481050,Apartment,2,2,824,999.0,2,0.0,1,0,1,0,0
31,Immediate,2000-01-01,Calgary,South Calgary,51.030291,-114.098947,/ab/calgary/rentals/apartment/1-bedroom/south-calgary/pet-friendly/2592,Apartment,2,2,464,950.0,1,0.0,1,0,1,0,0
62,Immediate,2000-01-01,Calgary,Willow Park,50.962396,-114.058199,/ab/calgary/rentals/apartment/1-bedroom/willow-park/477696,Apartment,0,0,650,999.0,1,0.0,1,0,1,0,0
63,Immediate,2000-01-01,Calgary,Lower Mount Royal,51.036402,-114.093876,/ab/calgary/rentals/apartment/1-bedroom/lower-mount-royal/pet-friendly/22158,Apartment,2,0,620,1000.0,1,0.0,1,0,1,0,0
93,NaN,3099-01-01,Calgary,Downtown,51.047748,-114.077661,/ab/calgary/rentals/apartment/1-bedroom/downtown/pet-friendly/14548,Apartment,2,0,490-652,975.0,1,0.0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,Immediate,2000-01-01,Calgary,Livingston,51.184966,-114.074045,/ab/calgary/rentals/townhouse/1-bedroom/livingston/pet-friendly/482132,Townhouse,2,2,505,2300.0,3,2.5,0,0,0,0,0
1937,Immediate,2000-01-01,Calgary,Victoria Park,51.039853,-114.066519,/ab/calgary/rentals/apartment/1-bedroom/victoria-park/pet-friendly/368446,Townhouse,2,2,1491,2800.0,0,2.0,1,0,1,0,0
1940,Apr 15,2022-04-15,Calgary,Thorncliffe,51.108820,-114.061987,/ab/calgary/rentals/townhouse/2-bedrooms/thorncliffe/pet-friendly/27543,Townhouse,2,2,982,1839.0,3,0.0,1,0,1,0,0
1972,May 01,2022-05-01,Calgary,Capitol Hill,51.073473,-114.093629,/ab/calgary/rentals/townhouse/3-bedrooms/capitol-hill/pet-friendly/469604,Townhouse,2,2,1502,3650.0,0,3.5,0,0,0,0,0


In [51]:
df.columns

Index(['rented', 'availability', 'a', 'city', 'community', 'latitude',
       'longitude', 'link', 'type', 'price', 'beds', 'sq_feet', 'baths',
       'cats', 'dogs', 'sq_feet2', 'price2', 'beds2', 'baths2', 'utility_heat',
       'utility_electricity', 'utility_water', 'utility_cable',
       'utility_internet'],
      dtype='object')

In [53]:
len(df[(~df.price2.isna())&(~df.sq_feet2.isna())&((~df.beds.isna()))&(~df.baths2.isna())])

180

In [59]:
df_second

,rented,availability,a,city,community,latitude,longitude,link,type,cats,dogs,sq_feet2,price2,beds2,baths2,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
7,Not-Rented,Immediate,2000-01-01,Calgary,Braeside,50.962467,-114.109704,/ab/calgary/rentals/apartment/1-bedroom/braeside/pet-friendly/481050,Apartment,2,2,824,999.0,2,0.0,1,0,1,0,0
31,Not-Rented,Immediate,2000-01-01,Calgary,South Calgary,51.030291,-114.098947,/ab/calgary/rentals/apartment/1-bedroom/south-calgary/pet-friendly/2592,Apartment,2,2,464,950.0,1,0.0,1,0,1,0,0
62,Not-Rented,Immediate,2000-01-01,Calgary,Willow Park,50.962396,-114.058199,/ab/calgary/rentals/apartment/1-bedroom/willow-park/477696,Apartment,0,0,650,999.0,1,0.0,1,0,1,0,0
63,Not-Rented,Immediate,2000-01-01,Calgary,Lower Mount Royal,51.036402,-114.093876,/ab/calgary/rentals/apartment/1-bedroom/lower-mount-royal/pet-friendly/22158,Apartment,2,0,620,1000.0,1,0.0,1,0,1,0,0
93,Not-Rented,NaN,3099-01-01,Calgary,Downtown,51.047748,-114.077661,/ab/calgary/rentals/apartment/1-bedroom/downtown/pet-friendly/14548,Apartment,2,0,490-652,975.0,1,0.0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,Not-Rented,Immediate,2000-01-01,Calgary,Livingston,51.184966,-114.074045,/ab/calgary/rentals/townhouse/1-bedroom/livingston/pet-friendly/482132,Townhouse,2,2,505,2300.0,3,2.5,0,0,0,0,0
1937,Not-Rented,Immediate,2000-01-01,Calgary,Victoria Park,51.039853,-114.066519,/ab/calgary/rentals/apartment/1-bedroom/victoria-park/pet-friendly/368446,Townhouse,2,2,1491,2800.0,0,2.0,1,0,1,0,0
1940,Not-Rented,Apr 15,2022-04-15,Calgary,Thorncliffe,51.108820,-114.061987,/ab/calgary/rentals/townhouse/2-bedrooms/thorncliffe/pet-friendly/27543,Townhouse,2,2,982,1839.0,3,0.0,1,0,1,0,0
1972,Not-Rented,May 01,2022-05-01,Calgary,Capitol Hill,51.073473,-114.093629,/ab/calgary/rentals/townhouse/3-bedrooms/capitol-hill/pet-friendly/469604,Townhouse,2,2,1502,3650.0,0,3.5,0,0,0,0,0


In [29]:
def clean_undefined_dates(df, null_substitute = 'call for availability'):
    for i in range(len(df)):
        try: 
            df.loc[i, 'a'] = pd.to_datetime(df.loc[i, 'a'], format='%Y-%m-%d')
        except: 
            df.loc[i, 'a'] = null_substitute
            df.loc[i, 'availability'] = null_substitute
    df.reset_index(inplace=True)
    return df
    
df = clean_undefined_dates(df)

In [35]:
df[df.sq_feet.isna()][['sq_feet', 'sq_feet2', 'link', 'type', 'price']].head()

,sq_feet,sq_feet2,link,type,price
0,NaN,NaN,/ab/calgary/rentals/apartment/1-bedroom/manchester/468894,Apartment,580.0
1,NaN,NaN,/ab/calgary/rentals/apartment/studio/downtown/472999,Apartment,580.0
4,NaN,NaN,/ab/calgary/rentals/apartment/1-bedroom/downtown/477769,Apartment,960.0
6,NaN,NaN,/ab/calgary/rentals/apartment/1-bedroom/beltline/452842,Apartment,925.0
8,NaN,NaN,/ab/calgary/rentals/apartment/2-bedrooms/banff-trail/pet-friendly/403379,Apartment,940.0


In [43]:
df[['price', 'price2']]

,price,price2
0,999.0,NaN
1,974.0,999.0
2,899.0,950.0
3,849.0,999.0
4,800.0,1000.0
...,...,...
182,1245.0,2300.0
183,2700.0,2800.0
184,1567.0,1839.0
185,2925.0,3650.0


In [27]:
df[df.availability != 'Immediate'].availability.unique()
#df.loc[:, 'availability_month'] 

array(['May 01', 'Apr 15', 'Apr 30', 'call for availability', 'May 15',
       'Apr 07', 'Apr 17', 'Apr 20', 'Apr 21', 'Apr 28', 'May 06',
       'May 07', 'May 09', 'May 10', 'May 13', 'May 20', 'Jun 01',
       'Jun 15', 'Jul 01', 'Jul 15', 'Aug 01', 'Sep 01', 'Apr 13',
       'Apr 14', 'Jun 30', 'Nov 01', 'Apr 11', 'Apr 09', 'Apr 10',
       'Apr 12', 'Apr 16', 'May 26', 'Apr 08', 'May 05', 'May 31',
       'Jul 08', 'Oct 01', 'Apr 18', 'May 02', 'May 16', 'Jun 05',
       'Sep 24', 'Jun 20', 'Jun 22', 'Jun 02', 'Apr 25', 'May 03',
       'May 18', 'May 21', 'May 28', 'Jun 07', 'Jun 16'], dtype=object)